In [ ]:
from backtest_lib.engine.decision import *
from backtest_lib.engine import make_engine
from backtest_lib.engine.execute.perfect_world import *
from backtest_lib.engine.plan.perfect_world import *
from backtest_lib.portfolio import cash, uniform_portfolio
import random
import sys

logging.basicConfig(
    level=logging.DEBUG,
    stream=sys.stdout,
    format="[%(levelname)s] %(name)s: %(message)s",
    force=True,
)

universe = (
    "security1",
    "security2",
    "security3",
    "security4",
    "security5",
)

prices = make_universe_mapping(
    {sec: random.random() * 1000 for sec in universe},
    universe=universe,
    constructor_backend="polars",
)

print(f"Set prices to {tuple(prices.items())}")

portfolio = uniform_portfolio(universe, value=100000, backend="polars")

decision_equal = target_weights({"security1": 0.5, "security2": 0.5}) + reallocate(
    0.2,
    out_of=("security1", "security2"),
    into=("security3", "security4"),
    mode="equal_out_equal_in",
)


def strategy_pro_rata(universe, market, current_portfolio, ctx):
    return target_weights({"security1": 0.5, "security2": 0.5}) + reallocate(
        0.2,
        out_of=("security1", "security2"),
        into=("security3", "security4"),
        mode=ReallocationMode.PRO_RATA_OUT_EQUAL_IN,
    )


def strategy_equal(universe, market, current_portfolio, ctx):
    return target_weights({"security1": 0.5, "security2": 0.5}) + reallocate(
        0.2,
        out_of=("security1", "security2"),
        into=("security3", "security4"),
        mode="equal_out_equal_in",
    )


planner = PerfectWorldPlanGenerator()
executor = PerfectWorldPlanExecutor(backend="polars", security_alignment=universe)

engine = make_engine(planner, executor, universe)

In [ ]:
plan = planner.generate_plan(decision_equal, prices)
execution_result = executor.execute_plan(
    plan=plan, portfolio=portfolio, prices=prices, market=None
)
assert list(execution_result.after.holdings.items()) == [
    ("security1", 0.4),
    ("security2", 0.4),
    ("security3", 0.1),
    ("security4", 0.1),
    ("security5", 0.0),
]

In [ ]:
execution_result_pro_rata = engine.execute_strategy(
    strategy=strategy_pro_rata,
    portfolio=portfolio,
    market=None,
    ctx=None,
    prices=prices,
)
execution_result_pro_rata.after.holdings

In [ ]:
execution_result_equal = engine.execute_strategy(
    strategy=strategy_equal, portfolio=portfolio, market=None, ctx=None, prices=prices
)
execution_result_equal.after.holdings